# Classification using pretrained, well-known models
This notebook aims to create a set of benchmarks for the project, using well-known, thoroughly studied models.
Either with pretrained weights, or training with new data.

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import master_data_functions.functions as dfunc
from keras.utils import to_categorical

# silence deprecation warnings from tensorflow
import warnings
warnings.simplefilter('ignore', FutureWarning)

# File import
# Sample filenames are:
# CeBr10kSingle_1.txt -> single events, 
# CeBr10kSingle_2.txt -> single events
# CeBr10k_1.txt -> mixed single and double events 
# CeBr10.txt -> small file of 10 samples

keywords = {
    "single_1": "CeBr10kSingle_1",
    "single_2": "CeBr10kSingle_2",
    "mix": "CeBr10k_1",
    "small": "CeBr10"}
    
dataset = "mix"
data = dfunc.import_data("sample")
images = data[keywords[dataset]]["images"]
energies = data[keywords[dataset]]["energies"]
positions = data[keywords[dataset]]["positions"]
labels = to_categorical(data[keywords[dataset]]["labels"])
n_classes = labels.shape[1]


print("Number of classes: {}".format(n_classes))
print("Energies shape: {}".format(energies.shape))
print("Positions shape: {}".format(positions.shape))
print("Labels shape: {}".format(labels.shape))
      

Using TensorFlow backend.
/home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be unde

Number of classes: 2
Energies shape: (10000, 2)
Positions shape: (10000, 4)
Labels shape: (10000, 2)


In [2]:
# VGG16 expects 3 channels. Solving this by concatenating the image data 
# to itself, to form three identical channels

images = np.concatenate((images, images, images), axis=3)

print("Image data shape: {}".format(images.shape))

Image data shape: (10000, 16, 16, 3)



## Set up training and test data

In [3]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2)
print("Training and test data shapes:")
print("x_train: {}".format(x_train.shape))
print("x_test: {}".format(x_test.shape))
print("y_train: {}".format(y_train.shape))
print("y_test: {}".format(y_test.shape))




Training and test data shapes:
x_train: (8000, 16, 16, 3)
x_test: (2000, 16, 16, 3)
y_train: (8000, 2)
y_test: (2000, 2)


## VGG-16


In [4]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False, classes=2)



W0811 15:05:47.903379 139808854820480 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 15:05:47.929760 139808854820480 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 15:05:47.939982 139808854820480 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 15:05:47.995432 139808854820480 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/n

In [26]:
# Set new config for input layer
model.summary()
import keras
from keras.engine.input_layer import Input
new_input = Input(shape=(0, 16, 16, 3))
new_input_shape=(None, 16, 16, 3)
model._layers[0].batch_input_shape = new_input_shape

# rebuild model architecture by exporting and importing via json
new_model = keras.models.model_from_json(model.to_json())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'block5_pool_2/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,512].